# 11.3: Cubic Spline Interpolation

## What Are Splines?

Splines are piecewise polynomial interpolants. For example, 
* a *linear spline* is simply just a spline connecting all the dots with the datapoints
    * not smooth, so can't take derivatives at points where you shift "pieces"
* a *quadratic spline* connects the dots with either concave up/concave down parabolas
    * more smooth than linear splines but can still have corners
* a *cubic spline* is arguably the best spline because it creates a very smooth curve, eliminating all corner problems

## Definition of Cubic Spline Interpolation

Given a dataset $\{(x_i,y_i)\}_{i=0}^n$ where $x_0 < x_1 < \cdots < x_n$. We will assume $y_i = f(x_i)$ for some unknown $f$. Then the interpolant will be a piecewise function $v(x)$ which is (if cubic)

* $s_0(x) = a_0 + b_0(x-x_0) + c_0(x-x_0)^2 + d_0(x-x_0)^3 \:\: x_0 \le x \le x_1$
* $s_1(x) = a_1 + b_1(x-x_1) + c_1(x-x_1)^2 + d_1(x-x_1)^3 \:\: x_1 \le x \le x_2$
* ...
* $s_{n-1}(x) = a_{n-1} + b_{n-1}(x-x_{n-1}) + c_{n-1}(x-x_{n-1})^2 + d_{n-1}(x-x_{n-1})^3 \:\: x_0 \le x \le x_1$


There are some requirements that splines follow:

> **Require the interpolating Condition**

* The left endpoint is $s_i(x_i) = y_i$ for $i=0,\cdots, n-1$ $\implies$ $a_i+b_i(x_i-x_i)+c_i(x_i-x_i)^2+d_i(x_i-x_i) = y_i \implies a_i = y_i$
* The right endpoints is $s_i(x_{i+1}) = y_{i+1} \implies a_i + b_i(x_{i+1}-x_i) + c_i(x_{i+1}-x_i)^2 + d_i(x_{i+1}-x_i)^3 = y_i \implies a_i + b_ih_i + c_ih_i^2 + d_ih_i^3 = y_{i+1}$
* There are $n$ of these equations


> **Require Smoothness**
* $s_i'(x_{i+1}) = s_{i+1}'(x_{i+1})$ from $i=0,\cdots, n-2$
    * $s_i'(x) = b_i + 2c_i(x-x_i) + 3d_i(x-x_i)^2$
    * $s_i'(x_{i+1}) = s_{i+1}'(x_{i+1}) \implies b_i + 2c_i(x_{i+1}-x_i)+3d_i(x_{i+1}-x_i)^2=b_{i+1} + 2c_{i+1}(x_{i+1}-x_{i+1}) + 3d_{i+1}(x_{i+1}-x_{i+1})^2$
   * There are $n-1$ of these equations
* $s_i''(x_{i+1}) = s''(x_{i+1})$
    * $s_i''(x_{i+1}) = 2c_i + 6d_i(x-x_i)$
    * $c_i + 3d_ib_i - c_{i+1} = 0$
    * There are $n-1$ of these
    
> **Boundary Conditions**

* "Natural/Free" boundary conditions: $s_0''(x_0) = 0$ and $s_{n-1}''(x_n)=0$
* This means that $c_0=0$ and $c_{n-1}+3d_{n-1}h_{n-1}=0$

## Example: Cubic Spline Through Two Data Points

Suppose we have a dataset containing the points (0,2), (1,4), (2,1). We need to find $a_0,b_0,c_0,d_0,a_1,b_1,c_1,d_1$. 

Let's start with our interpolating conditions:

* $a_0 = y_0 \implies a_0 = 2$
* $a_1 = y_1 \implies a_1 = 4$
* $a_0+b_0+c_0+d_0=4$
* $a_1+b_1+c_1+d_1=4$

Now let's check our smoothness conditions:
* $b_0 + 2c_0h_0 + 3d_0h_0^2-b_1 = 0$ but since $h_0 = 1$ we have $b_0 + 2c_0 + 3d_0-b_1 = 0$ so this is our first derivative
* Our second derivative looks like $c_0 + 3d_0 - c_1 = 0$ since $h_1=1$

Now let's check our boundary conditions:
* $c_0 = 0$
* $c_1 + 3_d1 = 0$

So our final $8 \times 8$ system looks like the following:

$$
\begin{aligned}
    \left[1 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\ 
          0 & 0 & 0 & 0 & 1 & 0 & 0 & 0\\ 
          1 & 1 & 1 & 1 & 0 & 0 & 0 & 0\\ 
          0 & 0 & 0 & 0 & 1 & 1 & 1 & 1\\ 
          0 & 1 & 2 & 3 & 0 & -1 & 0 & 0\\ 
          0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 \\ 
          0 & 0 & 0 & 0 & 0 & 0 & 1 & 3\right] \left[\begin{array}{c}a_0\\ b_0\\ c_0 \\ d_0 \\ a_1 \\ b_1 \\ c_1 \\ d_1\end{array}\right] = \left[\begin{array}{c}2\\4\\4\\1\\0\\0\\0\\0\end{array}\right]
\end{aligned}
$$

If we solve this, then our final $v(x)$ will be as follows:
* $2+3.25(x-0) - 1.25(x-0)^3$ for $0 \le x \le 1$
* $4 - 0.5(x-1)-3.75(x-1)^2+1.25(x-1)^3$ for $1 \le x \le 2$